In [2]:
%load_ext autoreload
%autoreload 2

In [83]:
from harvai.qa_model import get_answer

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from contextvars import Context
from re import A
from webbrowser import get
from transformers import pipeline

from harvai.data import preprocessing_user_input
from harvai.nn_model import Nn_model
from harvai.bm25 import Bm25
from harvai.dpr import DPR
from harvai.embedding import Embedding

In [5]:
from harvai.qa_model import get_context

2022-06-23 19:12:43.883779: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-23 19:12:43.883898: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
INFO - haystack.document_stores.base -  Numba not found, replacing njit() with no-op implementation. Enable it with 'pip install numba'.
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.
WARNING - haystack -  Object '__file__' is imported through a deprecated path. Please check out 

### KNN

In [131]:
"Qu'est-ce que le permis de conduire catégorie B?"
"Quelle est la vitesse maximale maximale des cyclomoteurs?"
"Quelle est la vitesse maximale autorisée sur autoroute ?"

'Quelle est la vitesse maximale autorisée sur autoroute ?'

In [152]:
article_number = 5
question = "Quelle est la vitesse normale autorisée sur autoroute ?"

In [153]:
Context_KNN, ParseCon_KNN , ArtRef_KNN = get_context(question,"KNN",article_number,digits=False)
# Context_DPR, ParseCon_DPR , ArtRef_DPR = get_context(question,"DPR",article_number,digits=False)
Context_BM25, ParseCon_BM25 , ArtRef_BM25 = get_context(question,"BM25",article_number,digits=False)
Context_Emb, ParseCon_Emb , ArtRef_Emb = get_context(question,"Embedding",article_number,digits=False)

In [154]:
print(ArtRef_KNN)
print(ArtRef_BM25)
print(ArtRef_Emb)

['Article R413-19  ', 'Article R413-2   ', 'Article R413-10  ', 'Article R413-9   ', 'Article R413-5   ']
['Article R413-2   ', 'Article R413-19  ', 'Article R413-9   ', 'Article R413-5   ', 'Article R413-8   ']
['Article R419-2   ', 'Article R413-2   ', 'Article R413-9   ', 'Article R413-8   ', 'Article R411-23-1']


In [155]:
dict_KNN = dict(zip(ArtRef_KNN, ParseCon_KNN))
dict_BM25 = dict(zip(ArtRef_BM25, ParseCon_BM25))
dict_Emb = dict(zip(ArtRef_Emb, ParseCon_Emb))

In [156]:
common_elements = []
for element in ArtRef_KNN:
    if element in ArtRef_BM25 and element in ArtRef_Emb :
        if element not in common_elements :
            common_elements.append(element)
        else :
            continue
common_elements

['Article R413-2   ', 'Article R413-9   ']

In [157]:
common_articles = []
for Artref in common_elements :
    common_articles.append(dict_KNN[Artref])

In [158]:
context = str(common_articles[0])

In [159]:
model = pipeline('question-answering', model='etalab-ia/camembert-base-squadFR-fquad-piaf', tokenizer='etalab-ia/camembert-base-squadFR-fquad-piaf')  

In [160]:
reponse = model({ 'question': question, 'context': context })
reponse["answer"]

' 130 km/ h'

In [161]:
get_answer(question,"BM25",article_number,digits=False)

({'score': 0.8248544335365295,
  'start': 4141,
  'end': 4149,
  'answer': ' 90 km/h'},
 ['I.-Hors agglomération, la vitesse des véhicules est limitée à :   1° 130 km/ h sur les autoroutes. Toutefois, lorsqu une voirie appartenant au réseau routier national a fait l objet d un classement dans la catégorie des autoroutes en application du sixième alinéa de l article R. 122-1 du code de la voirie routière, la vitesse maximale autorisée demeure celle fixée antérieurement à ce classement ;   2° 110 km/ h sur les routes à deux chaussées séparées par un terre-plein central ;   3° 80 km/ h sur les autres routes. Toutefois, sur les sections de ces routes comportant au moins deux voies affectées à un même sens de circulation, la vitesse maximale est relevée à 90 km/ h sur ces seules voies. Ces sections font l objet d une signalisation routière dans les conditions prévues par l article R. 411-25.   II.-En cas de pluie ou d autres précipitations, ces vitesses maximales sont abaissées à :   1° 110

In [50]:
ArtRef_DPR

NameError: name 'ArtRef_DPR' is not defined